In [2]:
import requests
from bs4 import BeautifulSoup
import re
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [1]:
#나무위키 체인지스에서 첫번째 td의 첫번째 전부 넣어라.

source_url="https://namu.wiki/RecentChanges"

req=requests.get(source_url)
html=req.content
soup=BeautifulSoup(html,'lxml')
contents_table=soup.find(name="table")
table_body=contents_table.find(name="tbody")
table_rows=table_body.find_all(name="tr")

page_url_base="https://namu.wiki"
page_urls=[]

for index in range(0,len(table_rows)):
    first_td=table_rows[index].find_all('td')[0]
    td_url=first_td.find_all('a')
    if len(td_url)>0:
        page_url=page_url_base+td_url[0].get('href')
        page_urls.append(page_url)

page_urls=list(set(page_urls))


for page in page_urls[:2]:
    print(page)

req=requests.get(page_urls[0])
html=req.content
soup=BeautifulSoup(html,'lxml')
contents_table=soup.find(name="article")
title=contents_table.find_all('h1')[0]
category=contents_table.find_all('ul')[0]
content_paragraphs=contents_table.find_all(name="div",attrs={"class":"wiki-parap-graph"})
content_corpus_list=[]

#크롤링한 문서 정보 출력
for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus="".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

NameError: name 'requests' is not defined

In [4]:
#크롤링 웹데이터 가져오기
source_url="https://namu.wiki/RecentChanges"#크롤링 할 사이트 주소
#사이트의 html 구조에 기반해서 크롤링 수행
req=requests.get(source_url)
html=req.content
soup=BeautifulSoup(html,'lxml')
contents_table=soup.find(name="table")
table_body=contents_table.find(name="tbody")
table_rows=table_body.find_all(name="tr")

#a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성한다.
page_url_base="https://namu.wiki"
page_urls=[]
for index in range(0,len(table_rows)):
    first_td=table_rows[index].find_all('td')[0]
    td_url=first_td.find_all('a')
    if len(td_url)>0:
        page_url=page_url_base+td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)
page_urls=list(set(page_urls))

#크롤링한 데이터를 데이터 프레임으로 만들 준비
columns=['title','category','content_text']
df=pd.DataFrame(columns=columns)

#각 페이지 별 제목 카테고리 본문 정보를 데이터 프레임으로
for page_url in page_urls:
    req=requests.get(page_url)
    html=req.content
    soup=BeautifulSoup(html,'lxml')
    contents_table=soup.find(name='article')
    title=contents_table.find_all('h1')[0]
    category=contents_table.find_all('ul')[0]
    content_paragraphs=contents_table.find_all(name="div",attrs={"class":"wiki-paragraph"})
    
    content_corpus_list=[]
    #페이지 내 제목 정보에서 개행 문자를 제거
    #만약 없는 경우,빈 문자열로 대체
    if title is not None:
        row_title=title.text.replace("\n"," ")
    else:
        row_title=""
    
    #페이지 내 본문 정보에서 개행문자를 제거
    #만약 없으면 빈 문자열
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n"," "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
    
    if category is not None:
        row_category=category.text.replace("\n"," ")
    else:
        row_category=""
    
    row=[row_title,row_category,".join(content_corpus_list)"]
    series=pd.Series(row,index=df.columns)
    df=df.append(series,ignore_index=True)
df.head()

,title,category,content_text
0,파일:LucasUAir.gif,파일/게임,.join(content_corpus_list)
1,NC 다이노스/2021년/5월,NC 다이노스/2021년,.join(content_corpus_list)
2,호세 피렐라,베네수엘라의 야구 선수1989년 출생2014년 데뷔우투우타내야수외야수뉴욕 양키스/은...,.join(content_corpus_list)
3,번민,중국의 인물용어,.join(content_corpus_list)
4,Tower Defense Simulator/이벤트,Tower Defense Simulator,.join(content_corpus_list)


In [8]:
def text_cleaning(text):
    hangul=re.compile('[^ ㄱ-ㅣ가-힣]+')
    result=hangul.sub('',text)
    return result
print(text_cleaning(df['content_text'][0]))